In [2]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione RING()

In [24]:
function ring(r=1., R=2., angle=2*pi)
    function ring0(shape=[36, 1])
        V, CV = cuboidGrid(shape)
        CV = [[[u,v,w],[w,v,t]] for (u,v,w,t) in CV]
        CV = reduce(append!,CV)
        V = [angle/shape[1] 0;0 (R-r)/shape[2]]*V
        V = broadcast(+, V, [0, r])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u)] end, W)...)
        W, CW = simplifyCells(V, CV)
        CW = [cell for cell in CW if length(cell)==3]
        return W,CW
    end
    return ring0
end

ring (generic function with 4 methods)

In [25]:
@btime W,CW = Lar.ring()();

  263.288 μs (3395 allocations: 290.61 KiB)


In [4]:
@code_warntype Lar.ring()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#ring0#281"{Float64,Float64,Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(36, 1)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


In [ ]:
@benchmark Lar.ring()()

## Uso macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [5]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()   

2

In [21]:
function simplifyCellsRef(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
    
    for incell in CV
        outcell = Int64[]
        for v in incell
         vert = V[:,v]
         key = map(approxVal(PRECISION), vert)
            if vertDict[key]==0
                index += 1
                vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
         push!(FW, [Set(outcell)...])
    end
    return hcat(W...),FW
end

simplifyCellsRef (generic function with 1 method)

In [22]:
function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

funzioni di supporto 

Funzione rivisitata per applicarci le macro @threads e @inbounds , ciclo for esteso .

In [23]:
function ring1(r=1., R=2., angle=2*pi)
    function ring01(shape=[36, 1])
        V, CV = Lar.cuboidGrid(shape)
        CV = [[[u,v,w],[w,v,t]] for (u,v,w,t) in CV]
        CV = reduce(append!,CV)
        V = [angle/shape[1] 0;0 (R-r)/shape[2]]*V
        V = broadcast(+, V, [0, r])
        
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
         @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
       
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u)] end, W)...)
        W, CW = simplifyCellsRef(V, CV)
        filter!(cell -> length(cell)==3,CW)
        return W,CW
    end
    return ring01
end

@btime W,CW = ring1()();

  262.429 μs (3316 allocations: 281.59 KiB)


Prestazioni migliorate di poco . Ho usato le funzioni di supporto per poter poi modificare e interagire con la mia versione della funzione ring , ho cercato di utilizzare i threads quindi per farlo ho dovuto "esternare" il ciclo for , cosi facendo ho potuto usare la macro @threads . Di fatti i cambiamenti non sono efficienti e spesso meno comprensibili e piu disordinati ma solo in questo modo sono state possibili applicare le macro . Senza l'uso delle macro , con il seguente ciclo for ho notato che la velocità di esecuzione aumenta di molto ( si puo notare nel notebook "disk" , nel caso di disk ho deciso di non omettere nessun passaggio e far vedere tutta l'evoluzione del codice stampandone i risultati legati alla velocità di esecuzione . ) Questo messaggio è comune e sarà riportato alla fine di ogni notebook .